In [1]:
# 导入库
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from sklearn import preprocessing
from keras.models import load_model

In [2]:
df1 = pd.read_csv('/content/成交量.csv',encoding='gbk')
df2 = pd.read_csv('/content/振幅.csv',encoding='gbk')
df3 = pd.read_csv('/content/换手率.csv',encoding='gbk')
df4 = pd.read_csv('/content/涨跌幅.csv',encoding='gbk')
df5 = pd.read_csv('/content/最低价.csv',encoding='gbk')
df6 = pd.read_csv('/content/最高价.csv',encoding='gbk')
df7 = pd.read_csv('/content/收盘价.csv',encoding='gbk')
df8 = pd.read_csv('/content/第二天涨跌幅.csv',encoding='gbk')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# data
def StockData(i):
    data1 = df1.iloc[i,4:]
    data2 = df2.iloc[i,4:]
    data3 = df3.iloc[i,4:]
    data4 = df4.iloc[i,4:]
    data5 = df5.iloc[i,4:]
    data6 = df6.iloc[i,4:]
    data7 = df7.iloc[i,4:]
    data8 = df8.iloc[i,4:]

    data = pd.DataFrame({  
        '成交量': list((data1).fillna(np.mean(data1))),
        '振幅 ':  list((data2).fillna(np.mean(data2))),
        '换手率': list((data3).fillna(np.mean(data3))),
        '涨跌幅': list((data4).fillna(np.mean(data4))),
        '最低价': list((data5).fillna(np.mean(data5))),
        '最高价': list((data6).fillna(np.mean(data6))),
        '收盘价': list((data7).fillna(np.mean(data7))),
        '第二天涨跌幅': list((data8).fillna(np.mean(data8)))
    })

    return data


In [ ]:
data = StockData(1)

In [ ]:
data.head()

,成交量,振幅,换手率,涨跌幅,最低价,最高价,收盘价,第二天涨跌幅
0,8.847028e+06,3.007301,1.660862,0.046709,39.863269,41.053345,40.434453,0.047674
1,8.847028e+06,3.007301,1.660862,0.046709,39.863269,41.053345,40.434453,0.047674
2,8.847028e+06,3.007301,1.660862,0.046709,39.863269,41.053345,40.434453,0.047674
3,8.847028e+06,3.007301,1.660862,0.046709,39.863269,41.053345,40.434453,0.047674
4,8.847028e+06,3.007301,1.660862,0.046709,39.863269,41.053345,40.434453,0.047674


In [14]:
# LSTM 对每个Stock
def LstmModel(name, data, feanum, window, cut, d, epochs, batch_size):
    #进行归一化操作
    min_max_scaler = preprocessing.MinMaxScaler()
    df0 = min_max_scaler.fit_transform(data)
    df = pd.DataFrame(df0, columns=data.columns)

    # 这一部分在处理数据 将原始数据改造为LSTM网络的输入
    stock=df
    seq_len=window
    amount_of_features = len(stock.columns) # 有几列
    data = stock.values # pd.DataFrame(stock) 表格转化为矩阵
    sequence_length = seq_len + 1 # 序列长度+1
    result = []
    for index in range(len(data) - sequence_length):#循环 数据长度-时间窗长度 次
        result.append(data[index: index + sequence_length])#第i行到i+5
    result = np.array(result)#得到样本，样本形式为 window*feanum

    #分训练集测试集 最后cut个样本为测试集
    train = result[:-cut, :]
    x_train = train[:, :-1]
    y_train = train[:, -1][:,-1]
    x_test = result[-cut:, :-1]
    y_test = result[-cut:, -1][:,-1]
    X_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
    X_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))  

    #建立、训练模型过程
    model = Sequential()#建立层次模型
    model.add(LSTM(64, input_shape=(window, feanum), return_sequences=True))#建立LSTM层
    model.add(Dropout(d))#建立的遗忘层
    model.add(LSTM(16, input_shape=(window, feanum), return_sequences=False))#建立LSTM层
    model.add(Dropout(d))#建立的遗忘层
    model.add(Dense(4,kernel_initializer='uniform',activation='relu'))   #建立全连接层     
    model.add(Dense(1,kernel_initializer='uniform',activation='relu'))
    model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
    model.fit(X_train, y_train, epochs, batch_size) #训练模型nb_epoch次

    model.save("/content/Models/"+str(name)+'.h5')

    # 在测试集上的预测
    y_test_predict=model.predict(X_test)[:,0]
    return y_test_predict

In [ ]:
predicted = LstmModel("1",data, feanum=data.shape[1],window=5,cut=1,d=0.0001,epochs=500,batch_size=256)


Epoch 1/256
5/5 [==============================] - 0s 5ms/step - loss: 0.2965 - accuracy: 4.1254e-04
Epoch 2/256
5/5 [==============================] - 0s 5ms/step - loss: 0.2908 - accuracy: 4.1254e-04
Epoch 3/256
5/5 [==============================] - 0s 4ms/step - loss: 0.2840 - accuracy: 4.1254e-04
Epoch 4/256
5/5 [==============================] - 0s 5ms/step - loss: 0.2761 - accuracy: 4.1254e-04
Epoch 5/256
5/5 [==============================] - 0s 5ms/step - loss: 0.2661 - accuracy: 4.1254e-04
Epoch 6/256
5/5 [==============================] - 0s 4ms/step - loss: 0.2525 - accuracy: 4.1254e-04
Epoch 7/256
5/5 [==============================] - 0s 5ms/step - loss: 0.2326 - accuracy: 4.1254e-04
Epoch 8/256
5/5 [==============================] - 0s 5ms/step - loss: 0.2048 - accuracy: 4.1254e-04
Epoch 9/256
5/5 [==============================] - 0s 4ms/step - loss: 0.1707 - accuracy: 4.1254e-04
Epoch 10/256
5/5 [==============================] - 0s 5ms/step - loss: 0.1349 - accuracy: 

In [ ]:
float(predicted)

0.5440482497215271

In [15]:
# 运行
StockNum = df1.shape[0]
StockName = []
PredictReturn = []
StockNum

504

In [16]:
StockNum/20

25.2

In [57]:
range((k*24),StockNum)

range(480, 504)

In [58]:
len(StockName)

500

In [59]:
k = 20
for i in range((k*24),StockNum):
  data = StockData(i)
  # 只预测一个
  predicted = LstmModel(str(i+1),data, feanum=data.shape[1],window=5,cut=1,d=0.0001,epochs=500,batch_size=256)
  StockName.append(i)
  PredictReturn.append(float(predicted))

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch 66/256
5/5 [==============================] - 0s 5ms/step - loss: 0.0025 - accuracy: 4.1254e-04
Epoch 67/256
5/5 [==============================] - 0s 7ms/step - loss: 0.0025 - accuracy: 4.1254e-04
Epoch 68/256
5/5 [==============================] - 0s 6ms/step - loss: 0.0025 - accuracy: 4.1254e-04
Epoch 69/256
5/5 [==============================] - 0s 5ms/step - loss: 0.0025 - accuracy: 4.1254e-04
Epoch 70/256
5/5 [==============================] - 0s 5ms/step - loss: 0.0025 - accuracy: 4.1254e-04
Epoch 71/256
5/5 [==============================] - 0s 7ms/step - loss: 0.0025 - accuracy: 4.1254e-04
Epoch 72/256
5/5 [==============================] - 0s 6ms/step - loss: 0.0025 - accuracy: 4.1254e-04
Epoch 73/256
5/5 [==============================] - 0s 5ms/step - loss: 0.0025 - accuracy: 4.1254e-04
Epoch 74/256
5/5 [==============================] - 0s 7ms/step - loss: 0.0025 - accuracy: 4.1254e-04
Epoch 75/256
5/5 [==============================] - 0s 

In [62]:
result = pd.DataFrame({  
        '股票代码': [i+1 for i in list(StockName)],
        '涨跌幅 ': PredictReturn
    })

In [63]:
result

,股票代码,涨跌幅
0,1,0.487857
1,2,0.493052
2,3,0.553116
3,4,0.000000
4,5,0.000000
...,...,...
519,500,0.000000
520,501,0.550901
521,502,0.584607
522,503,0.495896


In [ ]:
result.sort_values(by='涨跌幅',ascending = False)

KeyError: ignored

In [ ]:
from google.colab import files
files.download('a.csv')

In [64]:
import os, tarfile
 
import os
from google.colab import files

In [65]:
def make_targz_one_by_one(output_filename, source_dir):
  tar = tarfile.open(output_filename,"w")
  for root,dir_name,files_list in os.walk(source_dir):
    for file in files_list:
      pathfile = os.path.join(root, file)
      tar.add(pathfile)
  tar.close()
 
  files.download(output_filename)

In [67]:

make_targz_one_by_one('AllModels.tar', '/content/Models')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>